# Implement GLM and Contrast Function
A generalized linear model is used to find where brain activity changes given different tasks
The contrast function compares the brain activity between target and general task.

We are interested in the following contrast:
- Differentiate brain region activity between target and general task during audio condition
- Differentiate brain region activity between target and general task during visual condition
- Differentiate brain region activity between audio and visual condition after contrast is applied to both conditions

In [2]:
# import dependencies
from nilearn.glm.first_level import FirstLevelModel
import numpy as np

In [4]:
# Implement glm base model
def fit_glm(fmri_data, events_data, tr):
    """ 
    Fit a glm to the fmri data
    """
    model = FirstLevelModel(t_r = tr)
    glm = model.fit(fmri_data, events = events_data)
    return glm

In [ ]:
# Categorize stimuli into target and general task
def categorize_stimuli(events_data):
    """ 
    Categorize stimuli based on the filename patterns
    """
    # Extra

In [ ]:
# Implement contrast function
def contrast_glm_stimuli(glm, contrast_def = {'target': 1, 'general': -1}): 
    """ 
    Apply a contrast to the glm results
    """
    contrast = glm.compute_contrast(contrast_def)
    return contrast

def contrast_glm_conditions(glm, contrast_def = {'audio': 1, 'visual': -1}):
    """ 
    Apply a contrast to the glm results
    """
    contrast = glm.compute_contrast(contrast_def)
    return contrast